In [1]:
# !wget https://huggingface.co/datasets/mesolitica/chatgpt-news-extractive-qa/resolve/main/news-extractive-qa.json

In [12]:
import json
import random
import re

def clean(string):
    string = re.sub('[^A-Za-z ]+', ' ', string.lower())
    string = re.sub(r'[ ]+', ' ', string).strip()
    return string

def overlap(string1, string2):
    l = set([w for w in clean(string1).split() if len(w) > 2])
    r = set([w for w in clean(string2).split() if len(w) > 2])
    return len(l & r) / len(l)

In [3]:
with open('news-extractive-qa.json') as fopen:
    data = json.load(fopen)

In [4]:
len(data)

63202

In [5]:
data[0]

['Kuala Terengganu: Kementerian Perdagangan Dalam Negeri, Koperasi dan Kepenggunaan (KPDNKK) Terengganu merampas baju dan seluar jenama tiruan dianggar bernilai RM10,000 dalam serbuan sempena Ramadan di Kampung Tiong, di sini, petang semalam.\n\nDalam serbuan kira-kira 4.30 petang itu, KPDNKK menahan tiga individu termasuk pemilik premis berusia 50-an untuk siasatan mengikut Seksyen 8(2)(b) Akta Perihal Dagangan 2011.\n\nPengarah KPDNKK Terengganu Azlan Abdul Samad berkata, serbuan dilakukan di tiga premis termasuk sebuah stor di kampung sama dalam operasi diketuai tiga pegawai dan sembilan anggota hasil maklumat dan intipan sejak 8 Mei lalu.\n\n"Antara jenama dirampas ialah Adidas, Guess, Gucci, Levi\'s dan Nike dengan bekalan dipercayai diperoleh dari tiga negara jiran.\n\n"Siasatan mendapati pemilik premis berkenaan yang sudah 25 tahun mengaut keuntungan mudah menjalankan perniagaan pakaian tiruan itu dipercayai menjadi pembekal kepada peniaga kecil termasuk di pasar malam di negeri

In [6]:
data_qa = []
for d in data:
    try:
        t = d[0]
        a = d[1]
        j_a = json.loads(d[1])
        data_qa.append(
            {
                'context': t,
                'qa': j_a
            }
        )
    except Exception as e:
        pass
    
len(data_qa)

63013

In [7]:
data_qa[0]

{'context': 'Kuala Terengganu: Kementerian Perdagangan Dalam Negeri, Koperasi dan Kepenggunaan (KPDNKK) Terengganu merampas baju dan seluar jenama tiruan dianggar bernilai RM10,000 dalam serbuan sempena Ramadan di Kampung Tiong, di sini, petang semalam.\n\nDalam serbuan kira-kira 4.30 petang itu, KPDNKK menahan tiga individu termasuk pemilik premis berusia 50-an untuk siasatan mengikut Seksyen 8(2)(b) Akta Perihal Dagangan 2011.\n\nPengarah KPDNKK Terengganu Azlan Abdul Samad berkata, serbuan dilakukan di tiga premis termasuk sebuah stor di kampung sama dalam operasi diketuai tiga pegawai dan sembilan anggota hasil maklumat dan intipan sejak 8 Mei lalu.\n\n"Antara jenama dirampas ialah Adidas, Guess, Gucci, Levi\'s dan Nike dengan bekalan dipercayai diperoleh dari tiga negara jiran.\n\n"Siasatan mendapati pemilik premis berkenaan yang sudah 25 tahun mengaut keuntungan mudah menjalankan perniagaan pakaian tiruan itu dipercayai menjadi pembekal kepada peniaga kecil termasuk di pasar mala

In [8]:
indices = {i for i in range(len(data_qa))}

In [9]:
i = 0
for s in data_qa[i]['qa']['soalan']:
    print(s)

{'soalan': 'Di mana serbuan tersebut dilakukan?', 'substring': 'Kuala Terengganu'}
{'soalan': 'Berapa nilai baju dan seluar jenama tiruan yang dirampas?', 'substring': 'RM10,000'}
{'soalan': 'Berapa individu yang ditahan dalam serbuan tersebut?', 'substring': 'tiga individu'}
{'soalan': 'Apakah jenama-jenama yang dirampas?', 'substring': "Adidas, Guess, Gucci, Levi's dan Nike"}
{'soalan': 'Berapa lama pemilik premis tersebut menjalankan perniagaan pakaian tiruan?', 'substring': '25 tahun'}


In [20]:
from tqdm import tqdm

pairs = []
for i in tqdm(range(len(data_qa))):
    query = data_qa[i]['context']
    
    for s in data_qa[i]['qa']['soalan']:
        pos = [query]
        
        try:
        
            sampled = random.sample(list(indices - {i}), 100)
            neg = []
            for k in sampled:
                try:
                    overlapped = overlap(query, data_qa[k]['context'])
                    if overlapped < 0.3:
                        neg.append(data_qa[k]['context'])
                    if len(neg) >= 5:
                        break
                except:
                    pass

            pairs.append({
                'query': s['soalan'],
                'pos': pos,
                'neg': neg
            })
        except:
            pass

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 63013/63013 [07:01<00:00, 149.60it/s]


In [21]:
len(pairs)

315069

In [25]:
pairs[0]

{'query': 'Di mana serbuan tersebut dilakukan?',
 'pos': ['Kuala Terengganu: Kementerian Perdagangan Dalam Negeri, Koperasi dan Kepenggunaan (KPDNKK) Terengganu merampas baju dan seluar jenama tiruan dianggar bernilai RM10,000 dalam serbuan sempena Ramadan di Kampung Tiong, di sini, petang semalam.\n\nDalam serbuan kira-kira 4.30 petang itu, KPDNKK menahan tiga individu termasuk pemilik premis berusia 50-an untuk siasatan mengikut Seksyen 8(2)(b) Akta Perihal Dagangan 2011.\n\nPengarah KPDNKK Terengganu Azlan Abdul Samad berkata, serbuan dilakukan di tiga premis termasuk sebuah stor di kampung sama dalam operasi diketuai tiga pegawai dan sembilan anggota hasil maklumat dan intipan sejak 8 Mei lalu.\n\n"Antara jenama dirampas ialah Adidas, Guess, Gucci, Levi\'s dan Nike dengan bekalan dipercayai diperoleh dari tiga negara jiran.\n\n"Siasatan mendapati pemilik premis berkenaan yang sudah 25 tahun mengaut keuntungan mudah menjalankan perniagaan pakaian tiruan itu dipercayai menjadi pembek

In [22]:
with open('news-qa-pair.jsonl', 'w') as fopen:
    for l in pairs:
        if not len(l['pos']) or not len(l['neg']):
            continue
        fopen.write(f'{json.dumps(l)}\n')

In [23]:
from huggingface_hub import HfApi
api = HfApi()
api.upload_file(
    path_or_fileobj='news-qa-pair.jsonl',
    path_in_repo='news-qa-pair.jsonl',
    repo_id='mesolitica/instructions-pair-mining',
    repo_type='dataset',
)

news-qa-pair.jsonl:   0%|          | 0.00/3.98G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/mesolitica/instructions-pair-mining/commit/d0d54a31d02682483d09c5668f6511c24d69b41a', commit_message='Upload news-qa-pair.jsonl with huggingface_hub', commit_description='', oid='d0d54a31d02682483d09c5668f6511c24d69b41a', pr_url=None, pr_revision=None, pr_num=None)